In [26]:
%pip install openpyxl
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
# Cargar el archivo Excel
# Asegúrate de que el archivo 'Excel.xlsx' esté en la misma carpeta
try:
    df = pd.read_excel("Excel (1).xlsx")
except:
    # Si estás usando el CSV generado por error o conversión
    df = pd.read_csv("Excel.xlsx - Hoja1.csv") # Ajusta el nombre si es necesario

# --- BLOQUE DE SIMULACIÓN (Borrar o ignorar cuando tengas datos reales) ---
# Verificamos si la suma de puntajes es 0 (el archivo está vacío de notas)
# --- BLOQUE DE SIMULACIÓN MODIFICADO (0 a 10) ---
if df[['A', 'B', 'C', 'D']].sum().sum() == 0:
    print("⚠️ Datos en 0 detectados. Generando puntajes de prueba (0-10)...")
    np.random.seed(42)
    # Cambiamos (50, 100) por (0, 11) -> El 11 es exclusivo, así genera hasta el 10
    for col in ['A', 'B', 'C', 'D']:
        df[col] = np.random.randint(0, 11, len(df))

print("Datos cargados (Primeras 5 filas):")
display(df.head())

Datos cargados (Primeras 5 filas):


,Nombres,A,B,C,D
0,"ACERO GARCIA, SAMUEL",5,2,7,4
1,"BARRAGAN PEREZ, JULIAN ANDRES",4,2,6,4
2,"BELLO ORTIZ, CARLOS DAVID",3,2,5,3
3,"BELTRAN URBINA, ANDRES CAMILO",4,3,3,5
4,"CARPINTERO GABANZO, LUISA FERNANDA",7,4,4,6


In [28]:
# Seleccionamos las columnas de puntajes
features = ['A', 'B', 'C', 'D']
X = df[features]

# Normalizamos los datos entre 0 y 1 para compararlos equitativamente
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Creamos un DataFrame temporal con los datos escalados
df_scaled = pd.DataFrame(X_scaled, columns=features)

# Determinamos el "Rol Natural" (donde tienen su puntaje normalizado más alto)
df['Rol_Principal'] = df_scaled.idxmax(axis=1)
df['Max_Score_Scaled'] = df_scaled.max(axis=1)

# Ordenamos a los estudiantes: damos prioridad a los que tienen un talento muy marcado (outliers)
# para asegurar que queden en su rol ideal antes de llenar huecos.
df_sorted = df.sort_values(by='Max_Score_Scaled', ascending=False).copy()

print("Análisis de roles completado.")
display(df_sorted[['Nombres', 'Rol_Principal', 'Max_Score_Scaled']].head())

Análisis de roles completado.


,Nombres,Rol_Principal,Max_Score_Scaled
8,"GIL GALLEGO, ALEJANDRO",B,1.0
10,"LACOUTURE DAZA, JUAN ANDRES",A,1.0
7,"CONEJO GOMEZ, ANDREY ESTEBAN",B,1.0
5,"CARRILLO PIÑEROS, SANTIAGO",A,1.0
19,"VERGARA SUAREZ, JONATAN DAVID",B,1.0


In [29]:
def crear_grupos_inteligentes(df_input, features=['A', 'B', 'C', 'D']):
    # Trabajamos con una copia para no afectar el original
    pendientes = df_input.to_dict('records')
    
    num_grupos = len(pendientes) // 4
    grupos = [{k: None for k in features} for _ in range(num_grupos)]
    
    # Lista para quienes no logramos asignar en la primera ronda
    sin_asignar = []
    
    # --- RONDA 1: Asignación Preferente ---
    # Intentamos colocar a cada estudiante en su mejor rol
    for estudiante in pendientes:
        rol_ideal = estudiante['Rol_Principal']
        asignado = False
        
        # Buscamos el primer grupo que tenga ese espacio vacío
        for grupo in grupos:
            if grupo[rol_ideal] is None:
                grupo[rol_ideal] = estudiante
                asignado = True
                break
        
        if not asignado:
            sin_asignar.append(estudiante)
    
    # --- RONDA 2: Balanceo y Relleno (Backfilling) ---
    # Para los estudiantes que "sobraron" (porque su rol ideal estaba lleno),
    # buscamos su segunda o tercera mejor habilidad disponible.
    for estudiante in sin_asignar:
        # Obtenemos sus puntajes originales
        puntajes = {k: estudiante[k] for k in features}
        # Ordenamos habilidades de mayor a menor puntaje
        habilidades_ordenadas = sorted(puntajes, key=puntajes.get, reverse=True)
        
        asignado_final = False
        for habilidad in habilidades_ordenadas:
            for grupo in grupos:
                if grupo[habilidad] is None:
                    grupo[habilidad] = estudiante
                    # Marcamos que fue asignado a un rol secundario
                    estudiante['Rol_Asignado_Final'] = habilidad 
                    asignado_final = True
                    break
            if asignado_final:
                break
    
    return grupos

# Ejecutamos el algoritmo
grupos_finales = crear_grupos_inteligentes(df_sorted)
print(f"Se han generado {len(grupos_finales)} grupos exitosamente.")

Se han generado 5 grupos exitosamente.


In [30]:
def crear_grupos_inteligentes(df_input, features=['A', 'B', 'C', 'D']):
    # Trabajamos con una copia para no afectar el original
    pendientes = df_input.to_dict('records')
    
    num_grupos = len(pendientes) // 4
    grupos = [{k: None for k in features} for _ in range(num_grupos)]
    
    # Lista para quienes no logramos asignar en la primera ronda
    sin_asignar = []
    
    # --- RONDA 1: Asignación Preferente ---
    # Intentamos colocar a cada estudiante en su mejor rol
    for estudiante in pendientes:
        rol_ideal = estudiante['Rol_Principal']
        asignado = False
        
        # Buscamos el primer grupo que tenga ese espacio vacío
        for grupo in grupos:
            if grupo[rol_ideal] is None:
                grupo[rol_ideal] = estudiante
                asignado = True
                break
        
        if not asignado:
            sin_asignar.append(estudiante)
    
    # --- RONDA 2: Balanceo y Relleno (Backfilling) ---
    # Para los estudiantes que "sobraron" (porque su rol ideal estaba lleno),
    # buscamos su segunda o tercera mejor habilidad disponible.
    for estudiante in sin_asignar:
        # Obtenemos sus puntajes originales
        puntajes = {k: estudiante[k] for k in features}
        # Ordenamos habilidades de mayor a menor puntaje
        habilidades_ordenadas = sorted(puntajes, key=puntajes.get, reverse=True)
        
        asignado_final = False
        for habilidad in habilidades_ordenadas:
            for grupo in grupos:
                if grupo[habilidad] is None:
                    grupo[habilidad] = estudiante
                    # Marcamos que fue asignado a un rol secundario
                    estudiante['Rol_Asignado_Final'] = habilidad 
                    asignado_final = True
                    break
            if asignado_final:
                break
    
    return grupos

# Ejecutamos el algoritmo
grupos_finales = crear_grupos_inteligentes(df_sorted)
print(f"Se han generado {len(grupos_finales)} grupos exitosamente.")

Se han generado 5 grupos exitosamente.


In [31]:
# Estructurar los datos para exportar
datos_exportar = []

print("="*40)
print("      GRUPOS FORMADOS")
print("="*40)

for i, grupo in enumerate(grupos_finales):
    print(f"\n📂 GRUPO {i+1}")
    for rol in ['A', 'B', 'C', 'D']:
        estudiante = grupo.get(rol)
        if estudiante:
            nombre = estudiante['Nombres']
            puntaje = estudiante[rol]
            rol_orig = estudiante['Rol_Principal']
            
            # Detalle: ¿Es su rol fuerte o le tocó cubrir hueco?
            tipo = "⭐ Experto" if rol == rol_orig else "🔄 Cobertura"
            
            print(f"   [{rol}] {nombre:<35} | Nota: {puntaje} ({tipo})")
            
            datos_exportar.append({
                'Grupo_Numero': i+1,
                'Rol_En_Grupo': rol,
                'Estudiante': nombre,
                'Puntaje_Habilidad': puntaje,
                'Tipo_Asignacion': tipo,
                'Mejor_Habilidad_Real': rol_orig
            })
        else:
            print(f"   [{rol}] --- VACÍO ---")

# Guardar en Excel
df_resultado = pd.DataFrame(datos_exportar)
df_resultado.to_excel("Grupos_Formados.xlsx", index=False)
print("\n✅ Archivo 'Grupos_Formados.xlsx' guardado exitosamente.")

      GRUPOS FORMADOS

📂 GRUPO 1
   [A] LACOUTURE DAZA, JUAN ANDRES         | Nota: 8 (⭐ Experto)
   [B] GIL GALLEGO, ALEJANDRO              | Nota: 8 (⭐ Experto)
   [C] VALLE MORENO, MARIANA               | Nota: 8 (⭐ Experto)
   [D] CASTELLANOS ESTUPIÑAN, SEBASTIAN    | Nota: 8 (⭐ Experto)

📂 GRUPO 2
   [A] CARRILLO PIÑEROS, SANTIAGO          | Nota: 8 (⭐ Experto)
   [B] CONEJO GOMEZ, ANDREY ESTEBAN        | Nota: 8 (⭐ Experto)
   [C] SANCHEZ GARCIA, ANDRES FELIPE       | Nota: 8 (⭐ Experto)
   [D] JEREZ RAMIREZ, MARTIN               | Nota: 7 (⭐ Experto)

📂 GRUPO 3
   [A] URREA LARA, DEIVID NICOLAS          | Nota: 8 (⭐ Experto)
   [B] VERGARA SUAREZ, JONATAN DAVID       | Nota: 8 (⭐ Experto)
   [C] ACERO GARCIA, SAMUEL                | Nota: 7 (⭐ Experto)
   [D] BELTRAN URBINA, ANDRES CAMILO       | Nota: 5 (⭐ Experto)

📂 GRUPO 4
   [A] CARPINTERO GABANZO, LUISA FERNANDA  | Nota: 7 (⭐ Experto)
   [B] VARGAS HENAO, HAROLD STIVEN         | Nota: 8 (⭐ Experto)
   [C] BARRAGAN PEREZ, J